# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 12 2022 10:01AM,241637,10,Enova-10278,Emerginnova,Released
1,Jul 12 2022 9:52AM,241636,20,ALUR921184581,Alumier Labs Inc.,Released
2,Jul 12 2022 9:50AM,241635,10,0085867712,ISDIN Corporation,Released
3,Jul 12 2022 9:50AM,241635,10,0085867709,ISDIN Corporation,Released
4,Jul 12 2022 9:50AM,241635,10,0085867731,ISDIN Corporation,Released
5,Jul 12 2022 9:50AM,241635,10,0085867733,ISDIN Corporation,Released
6,Jul 12 2022 9:50AM,241635,10,0085867735,ISDIN Corporation,Released
7,Jul 12 2022 9:50AM,241635,10,0085867737,ISDIN Corporation,Released
8,Jul 12 2022 9:50AM,241635,10,0085867789,ISDIN Corporation,Released
9,Jul 12 2022 9:50AM,241635,10,0085867792,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
33,241633,Released,15
34,241634,Released,1
35,241635,Released,11
36,241636,Released,1
37,241637,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
241633,NaN,15.0
241634,NaN,1.0
241635,NaN,11.0
241636,NaN,1.0
241637,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
241522,0.0,55.0
241523,0.0,63.0
241529,0.0,49.0
241560,13.0,9.0
241562,0.0,20.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
241522,0,55
241523,0,63
241529,0,49
241560,13,9
241562,0,20


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,241522,0,55
1,241523,0,63
2,241529,0,49
3,241560,13,9
4,241562,0,20


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,241522,,55
1,241523,,63
2,241529,,49
3,241560,13,9
4,241562,,20


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 12 2022 10:01AM,241637,10,Emerginnova
1,Jul 12 2022 9:52AM,241636,20,Alumier Labs Inc.
2,Jul 12 2022 9:50AM,241635,10,ISDIN Corporation
13,Jul 12 2022 9:40AM,241633,10,ISDIN Corporation
28,Jul 12 2022 9:40AM,241634,10,ISDIN Corporation
29,Jul 12 2022 9:29AM,241632,10,ISDIN Corporation
70,Jul 12 2022 9:24AM,241622,10,ISDIN Corporation
117,Jul 12 2022 9:18AM,241630,20,"Exact-Rx, Inc."
128,Jul 12 2022 9:17AM,241629,12,Hush Hush
131,Jul 12 2022 9:17AM,241628,19,"GCH Granules USA, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jul 12 2022 10:01AM,241637,10,Emerginnova,,1
1,Jul 12 2022 9:52AM,241636,20,Alumier Labs Inc.,,1
2,Jul 12 2022 9:50AM,241635,10,ISDIN Corporation,,11
3,Jul 12 2022 9:40AM,241633,10,ISDIN Corporation,,15
4,Jul 12 2022 9:40AM,241634,10,ISDIN Corporation,,1
5,Jul 12 2022 9:29AM,241632,10,ISDIN Corporation,,41
6,Jul 12 2022 9:24AM,241622,10,ISDIN Corporation,,47
7,Jul 12 2022 9:18AM,241630,20,"Exact-Rx, Inc.",,11
8,Jul 12 2022 9:17AM,241629,12,Hush Hush,2,1
9,Jul 12 2022 9:17AM,241628,19,"GCH Granules USA, Inc.",,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 12 2022 10:01AM,241637,10,Emerginnova,1,
1,Jul 12 2022 9:52AM,241636,20,Alumier Labs Inc.,1,
2,Jul 12 2022 9:50AM,241635,10,ISDIN Corporation,11,
3,Jul 12 2022 9:40AM,241633,10,ISDIN Corporation,15,
4,Jul 12 2022 9:40AM,241634,10,ISDIN Corporation,1,
5,Jul 12 2022 9:29AM,241632,10,ISDIN Corporation,41,
6,Jul 12 2022 9:24AM,241622,10,ISDIN Corporation,47,
7,Jul 12 2022 9:18AM,241630,20,"Exact-Rx, Inc.",11,
8,Jul 12 2022 9:17AM,241629,12,Hush Hush,1,2
9,Jul 12 2022 9:17AM,241628,19,"GCH Granules USA, Inc.",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 12 2022 10:01AM,241637,10,Emerginnova,1,
1,Jul 12 2022 9:52AM,241636,20,Alumier Labs Inc.,1,
2,Jul 12 2022 9:50AM,241635,10,ISDIN Corporation,11,
3,Jul 12 2022 9:40AM,241633,10,ISDIN Corporation,15,
4,Jul 12 2022 9:40AM,241634,10,ISDIN Corporation,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jul 12 2022 10:01AM,241637,10,Emerginnova,1.0,NaN
1,Jul 12 2022 9:52AM,241636,20,Alumier Labs Inc.,1.0,NaN
2,Jul 12 2022 9:50AM,241635,10,ISDIN Corporation,11.0,NaN
3,Jul 12 2022 9:40AM,241633,10,ISDIN Corporation,15.0,NaN
4,Jul 12 2022 9:40AM,241634,10,ISDIN Corporation,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 12 2022 10:01AM,241637,10,Emerginnova,1.0,0.0
1,Jul 12 2022 9:52AM,241636,20,Alumier Labs Inc.,1.0,0.0
2,Jul 12 2022 9:50AM,241635,10,ISDIN Corporation,11.0,0.0
3,Jul 12 2022 9:40AM,241633,10,ISDIN Corporation,15.0,0.0
4,Jul 12 2022 9:40AM,241634,10,ISDIN Corporation,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,4348801,467.0,53.0
12,241629,1.0,2.0
16,241601,1.0,0.0
19,1207997,3.0,5.0
20,1208013,60.0,13.0
21,483225,2.0,0.0
22,241623,0.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,4348801,467.0,53.0
1,12,241629,1.0,2.0
2,16,241601,1.0,0.0
3,19,1207997,3.0,5.0
4,20,1208013,60.0,13.0
5,21,483225,2.0,0.0
6,22,241623,0.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,467.0,53.0
1,12,1.0,2.0
2,16,1.0,0.0
3,19,3.0,5.0
4,20,60.0,13.0
5,21,2.0,0.0
6,22,0.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,467.0
1,12,Released,1.0
2,16,Released,1.0
3,19,Released,3.0
4,20,Released,60.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,19,20,21,22
Status,,,,,,,
Executing,53.0,2.0,0.0,5.0,13.0,0.0,1.0
Released,467.0,1.0,1.0,3.0,60.0,2.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,19,20,21,22
0,Executing,53.0,2.0,0.0,5.0,13.0,0.0,1.0
1,Released,467.0,1.0,1.0,3.0,60.0,2.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,19,20,21,22
0,Executing,53.0,2.0,0.0,5.0,13.0,0.0,1.0
1,Released,467.0,1.0,1.0,3.0,60.0,2.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()